[![Binder](https://mybinder.org/badge_logo.svg)](https://nbviewer.org/github/Sistemas-Multimedia/Sistemas-Multimedia.github.io/blob/master/contents/RGB_VQ/RGB_VQ.ipynb)

[![Colab](https://badgen.net/badge/Launch/on%20Google%20Colab/blue?icon=notebook)](https://colab.research.google.com/github/Sistemas-Multimedia/Sistemas-Multimedia.github.io/blob/master/contents/RGB_VQ/RGB_VQ.ipynb)

# [Vector Quantization (in the color domain) of a RGB image](https://scikit-learn.org/stable/auto_examples/cluster/plot_color_quantization.html#sphx-glr-auto-examples-cluster-plot-color-quantization-py)
Color VQ using [K-means](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans).

In [ ]:
%%bash
if [ -d "$HOME/repos" ]; then
    echo "\"$HOME/repos\" exists"
else
    mkdir ~/repos
    echo Created $HOME/repos
fi

In [ ]:
%%bash
if [ -d "$HOME/repos/scalar_quantization" ]; then
    cd $HOME/repos/scalar_quantization
    echo "$HOME/repos/scalar_quantization ... "
    git pull 
else
    cd $HOME/repos
    git clone https://github.com/vicente-gonzalez-ruiz/scalar_quantization.git
fi

In [ ]:
%%bash
if [ -d "$HOME/repos/image_IO" ]; then
    cd $HOME/repos/image_IO
    echo "$HOME/repos/image_IO ... "
    git pull 
else
    cd $HOME/repos
    git clone https://github.com/vicente-gonzalez-ruiz/image_IO.git
fi

In [ ]:
%%bash
if [ -d "$HOME/repos/MRVC" ]; then
    cd $HOME/repos/MRVC
    echo "$HOME/repos/MRVC ... "
    git pull 
else
    cd $HOME/repos
    git clone https://github.com/Sistemas-Multimedia/MRVC.git
fi

In [ ]:
!ln -sf ~/repos/information_theory/information.py .
!ln -sf ~/repos/information_theory/distortion.py .
!ln -sf ~/repos/image_IO/image_1.py .
!ln -sf ~/repos/image_IO/image_3.py .
!ln -sf ~/repos/image_IO/logging_config.py .

In [ ]:
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
from sklearn.utils import shuffle
import image_3 as RGB_image
import image_1 as gray_image
import distortion
import information
import os
import pylab
import gzip

## Configuration

In [ ]:
home = os.environ["HOME"]
fn = home + "/repos/MRVC/images/lena_color/"
n_clusters = 256  # Number of bins
range_of_N_bins = [1<<i for i in range(1, 8)]
#gray_image.write = gray_image.debug_write # faster, but lower compression
image_dtype = np.uint8 # For 8 bpp/component images
#image_dtype = np.uint16 # For 16 bpp/component images

#RGB_image.write = RGB_image.debug_write # Faster, but lower compression
RGB_image.write = information.write # The fastest, but returns only an estimation of the length
#gray_image.write = gray_image.debug_write # Faster, but lower compression
gray_image.write = information.write # The fastest, but returns only an estimation of the length

## Read the image and show it

In [ ]:
RGB_img = RGB_image.read(fn).astype(image_dtype)
RGB_image.show(RGB_img, fn + "000.png")

## Example

In [ ]:
# Create a K-means clustering tool
normalized_img = np.array(RGB_img,dtype=np.float64) / 255
w, h, d = shape_of_img = tuple(RGB_img.shape)
img_reshaped = normalized_img.reshape((w*h, d))
kmeans = KMeans(n_clusters=32, random_state=0)
some_samples = shuffle(img_reshaped, random_state=0, n_samples=1_000)

In [ ]:
# Find centroids in clusters (CPU intensive)
kmeans.fit(some_samples)

In [ ]:
# Quantize
centroids = kmeans.cluster_centers_
labels = kmeans.predict(img_reshaped)
VQ_img = centroids[labels].reshape(w, h, -1)
RGB_image.show(VQ_img, "Quantized Image using floating point pallete")

In [ ]:
print(centroids)

In [ ]:
centroids = (centroids * 255).astype(np.uint8)
print(centroids)

In [ ]:
VQ_img = centroids[labels].reshape(w, h, -1)
RGB_image.show(VQ_img, "Quantized Image using integer pallete")

In [ ]:
with gzip.GzipFile("/tmp/codebook.npy.gz", "w") as f:
    np.save(file=f, arr=centroids)
os.path.getsize("/tmp/codebook.npy.gz")

In [ ]:
with gzip.GzipFile("/tmp/codebook.npy.gz", "r") as f:
    centroids = np.load(f)
print(centroids)

## RD performance
The code-book is considered in the computation of the bit-rate.

In [ ]:
def RD_curve(img, range_of_N_bins):
    normalized_img = np.array(RGB_img,dtype=np.float64) / 255
    w, h, d = shape_of_img = tuple(RGB_img.shape)
    img_reshaped = normalized_img.reshape((w*h, d))
    points = []
    for n in range_of_N_bins:
        kmeans = KMeans(n_clusters=n, random_state=0)
        some_samples = shuffle(img_reshaped, random_state=0, n_samples=1_000)
        kmeans.fit(some_samples)
        centroids = kmeans.cluster_centers_
        labels = kmeans.predict(img_reshaped)
        k = labels.astype(np.uint8) # Up to 256 bins
        y = (centroids[k].reshape(w, h, -1) * 255).astype(np.uint8)
        print("Quantization indexes: ", np.unique(k))
        rate = RGB_image.write(k, "/tmp/" + str(n) + '_', 0)*8/(img.shape[0]*img.shape[1])
        print(img.size)
        with gzip.GzipFile("/tmp/codebook.npy.gz", "w") as f:
            np.save(file=f, arr=centroids)
        rate += (os.path.getsize("/tmp/codebook.npy.gz")*8/(img.shape[0]*img.shape[1]))
        _distortion = distortion.RMSE(img, y)
        plt.title(f"{n}")
        plt.imshow(y, cmap=plt.cm.gray, vmin=0, vmax=256)
        plt.show()
        points.append((rate, _distortion))
        print(f"n={n:>3}, rate={rate:>7} bits/pixel, distortion={_distortion:>6.1f}")
    return points

RD_points = RD_curve(RGB_img, range_of_N_bins)

In [ ]:
YCoCg_SQ = []
with open(f'../YCoCg_SQ/YCoCg_SQ.txt', 'r') as f:
    for line in f:
        BPP, RMSE = line.split('\t')
        YCoCg_SQ.append((float(BPP), float(RMSE)))

In [ ]:
pylab.figure(dpi=150)
#pylab.scatter(*zip(*RD_points), label=f"VQ+PNG", s=1, marker='.')
pylab.plot(*zip(*YCoCg_SQ), c='b', marker='o', label="Deadzone($\mathbf{\Delta}^{\mathrm{Y}}_i = \mathbf{\Delta}^{\mathrm{Co}}_i = \mathbf{\Delta}^{\mathrm{Cg}}_i$)+PNG", linestyle="dashed")
pylab.plot(*zip(*RD_points), c='m', marker='x', label="VQ($\mathrm{RGB}$)+PNG", linestyle="dotted")
pylab.title(fn)
pylab.xlabel("Bits/Pixel")
pylab.ylabel("RMSE")
pylab.legend(loc='upper right')
pylab.show()

In [ ]:
with open(f"RGB_VQ.txt", 'w') as f:
    for item in RD_points:
        f.write(f"{item[0]}\t{item[1]}\n")

In [ ]:
import time
while True:
    time.sleep(1)

## Ignore the rest ...

## Compare to SQ

In [ ]:
def load(path):
    curve = []
    with open(path, 'r') as f:
        for line in f:
            rate, _distortion = line.split('\t')
            curve.append((float(rate), float(_distortion)))
    return curve

In [ ]:
SQ_color = load("../RGB_SQ/RGB_SQ.txt")

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*RD_points), c='m', marker='x', label="VQ", linestyle="dotted")
pylab.plot(*zip(*SQ_color), c='k', marker='x', label="SQ", linestyle="dotted")
pylab.title(fn)
pylab.xlabel("Bits/Pixel")
pylab.ylabel("RMSE")
pylab.legend(loc='upper right')
#pylab.xscale("log")
#pylab.yscale("log")
pylab.show()

In [ ]:
import time
while True:
    time.sleep(1)

In [ ]:
if 'google.colab' in str(get_ipython()):
    %run ./deadzone.ipynb
    deadzone_RD = load("./dead-zone_RD_points.txt")
    print("loaded ./dead-zone_RD_points.txt")
elif not os.path.exists("../scalar_quantization/dead-zone_RD_points.txt"):
    %run ../scalar_quantization/deadzone.ipynb
    deadzone_RD = load("../scalar_quantization/dead-zone_RD_points.txt")
    print("loaded ../scalar_quantization/dead-zone_RD_points.txt after regenerating the file")
else:
    deadzone_RD = load("../scalar_quantization/dead-zone_RD_points.txt")
    print("loaded ../scalar_quantization/dead-zone_RD_points.txt without regenerating the file")